##### CAUSAL SELF-ATTENTION

In [21]:
import torch
inputs = torch.tensor(
[[[0.43, 0.15, 0.89], # Your
 [0.55, 0.87, 0.66], # journey
 [0.57, 0.85, 0.64], # starts
 [0.22, 0.58, 0.33], # with
 [0.77, 0.25, 0.10], # one
 [0.05, 0.80, 0.55]], # step
 [[0.43, 0.15, 0.89], # Your
 [0.55, 0.87, 0.66], # journey
 [0.57, 0.85, 0.64], # starts
 [0.22, 0.58, 0.33], # with
 [0.77, 0.25, 0.10], # one
 [0.05, 0.80, 0.55]]] # step
)
inputs.shape

torch.Size([2, 6, 3])

In [26]:
import torch.nn as nn

context_length = inputs.shape[1]
d_in = inputs.shape[-1]
d_out = 2
W_query = nn.Linear(d_in, d_out, bias = False)
W_key = nn.Linear(d_in, d_out, bias = False)
W_value = nn.Linear(d_in, d_out, bias = False)
dropout = nn.Dropout(0.5)
mask = torch.triu(torch.ones(context_length,context_length), diagonal = 1)

In [27]:
queries = W_key(inputs)
keys = W_key(inputs)
values = W_value(inputs)
queries.shape, keys.shape, values.shape

(torch.Size([2, 6, 2]), torch.Size([2, 6, 2]), torch.Size([2, 6, 2]))

In [33]:
attn_scores = queries @ keys.transpose(1,2)
attn_scores.masked_fill_(mask.bool(),-torch.inf)

tensor([[[0.7175,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.8337, 1.0516,   -inf,   -inf,   -inf,   -inf],
         [0.8237, 1.0358, 1.0204,   -inf,   -inf,   -inf],
         [0.4381, 0.5731, 0.5638, 0.3171,   -inf,   -inf],
         [0.4121, 0.4597, 0.4549, 0.2368, 0.2412,   -inf],
         [0.5663, 0.7649, 0.7516, 0.4284, 0.3005, 0.5847]],

        [[0.7175,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.8337, 1.0516,   -inf,   -inf,   -inf,   -inf],
         [0.8237, 1.0358, 1.0204,   -inf,   -inf,   -inf],
         [0.4381, 0.5731, 0.5638, 0.3171,   -inf,   -inf],
         [0.4121, 0.4597, 0.4549, 0.2368, 0.2412,   -inf],
         [0.5663, 0.7649, 0.7516, 0.4284, 0.3005, 0.5847]]],
       grad_fn=<MaskedFillBackward0>)

In [35]:
attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim = -1)
attn_weights

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4616, 0.5384, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3020, 0.3509, 0.3471, 0.0000, 0.0000, 0.0000],
         [0.2432, 0.2676, 0.2659, 0.2233, 0.0000, 0.0000],
         [0.2068, 0.2139, 0.2132, 0.1827, 0.1833, 0.0000],
         [0.1656, 0.1905, 0.1888, 0.1502, 0.1372, 0.1677]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4616, 0.5384, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3020, 0.3509, 0.3471, 0.0000, 0.0000, 0.0000],
         [0.2432, 0.2676, 0.2659, 0.2233, 0.0000, 0.0000],
         [0.2068, 0.2139, 0.2132, 0.1827, 0.1833, 0.0000],
         [0.1656, 0.1905, 0.1888, 0.1502, 0.1372, 0.1677]]],
       grad_fn=<SoftmaxBackward0>)

In [37]:
attn_weights = dropout(torch.softmax(attn_scores/keys.shape[-1]**0.5, dim = -1))
attn_weights

tensor([[[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 1.0769, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.6040, 0.0000, 0.6942, 0.0000, 0.0000, 0.0000],
         [0.4865, 0.0000, 0.5317, 0.4466, 0.0000, 0.0000],
         [0.4137, 0.4278, 0.4264, 0.3655, 0.0000, 0.0000],
         [0.3311, 0.0000, 0.3775, 0.3004, 0.0000, 0.3355]],

        [[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.9231, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.6040, 0.7018, 0.6942, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.5352, 0.0000, 0.4466, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.3666, 0.0000],
         [0.3311, 0.3811, 0.3775, 0.0000, 0.0000, 0.3355]]],
       grad_fn=<MulBackward0>)

In [40]:
context_vec = attn_weights @ values
context_vec, context_vec.shape

(tensor([[[-0.1178,  0.0542],
          [ 0.2136,  0.2514],
          [ 0.1003,  0.1752],
          [ 0.1386,  0.2044],
          [ 0.1957,  0.2655],
          [ 0.1548,  0.2119]],
 
         [[-0.1178,  0.0542],
          [-0.0544,  0.0250],
          [ 0.2395,  0.3390],
          [ 0.1694,  0.1945],
          [ 0.0340,  0.0293],
          [ 0.1879,  0.2541]]], grad_fn=<UnsafeViewBackward0>),
 torch.Size([2, 6, 2]))

##### MULTIHEADED SELF-ATTENTION

In [41]:
import torch
inputs = torch.tensor(
[[[0.43, 0.15, 0.89], # Your
 [0.55, 0.87, 0.66], # journey
 [0.57, 0.85, 0.64], # starts
 [0.22, 0.58, 0.33], # with
 [0.77, 0.25, 0.10], # one
 [0.05, 0.80, 0.55]], # step
 [[0.43, 0.15, 0.89], # Your
 [0.55, 0.87, 0.66], # journey
 [0.57, 0.85, 0.64], # starts
 [0.22, 0.58, 0.33], # with
 [0.77, 0.25, 0.10], # one
 [0.05, 0.80, 0.55]]] # step
)
inputs.shape

torch.Size([2, 6, 3])

In [42]:
import torch.nn as nn

context_length = inputs.shape[1]
d_in = inputs.shape[-1]
num_heads = 2
d_out = 2
head_dim = d_out // num_heads
W_query = nn.Linear(d_in, d_out, bias = False)
W_key = nn.Linear(d_in, d_out, bias = False)
W_value = nn.Linear(d_in, d_out, bias = False)
dropout = nn.Dropout(0.5)
out_proj = nn.Linear(d_out, d_out) #Optional
mask = torch.triu(torch.ones(context_length,context_length), diagonal = 1)

In [45]:
b, num_tokens, d_in = inputs.shape
queries = W_key(inputs)
keys = W_key(inputs)
values = W_value(inputs)
queries.shape, keys.shape, values.shape

(torch.Size([2, 6, 2]), torch.Size([2, 6, 2]), torch.Size([2, 6, 2]))

In [47]:
keys = keys.view(b, num_tokens, num_heads, head_dim)
values = values.view(b, num_tokens, num_heads,head_dim)
queries = queries.view(b, num_tokens, num_heads, head_dim)
queries.shape, keys.shape, values.shape #batch, num_tokens, num_heads, head_dim

(torch.Size([2, 6, 2, 1]), torch.Size([2, 6, 2, 1]), torch.Size([2, 6, 2, 1]))

In [48]:
keys = keys.transpose(1, 2)
values = values.transpose(1, 2)
queries = queries.transpose(1, 2)
queries.shape, keys.shape, values.shape #batch, num_heads, num_tokens, head_dim

(torch.Size([2, 2, 6, 1]), torch.Size([2, 2, 6, 1]), torch.Size([2, 2, 6, 1]))

In [49]:
attn_scores = queries @ keys.transpose(2,3)
attn_scores, attn_scores.shape

(tensor([[[[0.1908, 0.3380, 0.3303, 0.2040, 0.0988, 0.2898],
           [0.3380, 0.5989, 0.5853, 0.3614, 0.1752, 0.5135],
           [0.3303, 0.5853, 0.5720, 0.3532, 0.1712, 0.5019],
           [0.2040, 0.3614, 0.3532, 0.2181, 0.1057, 0.3099],
           [0.0988, 0.1752, 0.1712, 0.1057, 0.0512, 0.1502],
           [0.2898, 0.5135, 0.5019, 0.3099, 0.1502, 0.4403]],
 
          [[0.4089, 0.6302, 0.6246, 0.3471, 0.3468, 0.4247],
           [0.6302, 0.9714, 0.9627, 0.5350, 0.5345, 0.6547],
           [0.6246, 0.9627, 0.9540, 0.5302, 0.5297, 0.6488],
           [0.3471, 0.5350, 0.5302, 0.2946, 0.2944, 0.3605],
           [0.3468, 0.5345, 0.5297, 0.2944, 0.2941, 0.3602],
           [0.4247, 0.6547, 0.6488, 0.3605, 0.3602, 0.4412]]],
 
 
         [[[0.1908, 0.3380, 0.3303, 0.2040, 0.0988, 0.2898],
           [0.3380, 0.5989, 0.5853, 0.3614, 0.1752, 0.5135],
           [0.3303, 0.5853, 0.5720, 0.3532, 0.1712, 0.5019],
           [0.2040, 0.3614, 0.3532, 0.2181, 0.1057, 0.3099],
           [0.0

In [50]:
attn_scores.masked_fill_(
            mask.bool()[:num_tokens,:num_tokens], -torch.inf)

tensor([[[[0.1908,   -inf,   -inf,   -inf,   -inf,   -inf],
          [0.3380, 0.5989,   -inf,   -inf,   -inf,   -inf],
          [0.3303, 0.5853, 0.5720,   -inf,   -inf,   -inf],
          [0.2040, 0.3614, 0.3532, 0.2181,   -inf,   -inf],
          [0.0988, 0.1752, 0.1712, 0.1057, 0.0512,   -inf],
          [0.2898, 0.5135, 0.5019, 0.3099, 0.1502, 0.4403]],

         [[0.4089,   -inf,   -inf,   -inf,   -inf,   -inf],
          [0.6302, 0.9714,   -inf,   -inf,   -inf,   -inf],
          [0.6246, 0.9627, 0.9540,   -inf,   -inf,   -inf],
          [0.3471, 0.5350, 0.5302, 0.2946,   -inf,   -inf],
          [0.3468, 0.5345, 0.5297, 0.2944, 0.2941,   -inf],
          [0.4247, 0.6547, 0.6488, 0.3605, 0.3602, 0.4412]]],


        [[[0.1908,   -inf,   -inf,   -inf,   -inf,   -inf],
          [0.3380, 0.5989,   -inf,   -inf,   -inf,   -inf],
          [0.3303, 0.5853, 0.5720,   -inf,   -inf,   -inf],
          [0.2040, 0.3614, 0.3532, 0.2181,   -inf,   -inf],
          [0.0988, 0.1752, 0.1712,

In [51]:
attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim = -1)
attn_weights = dropout(attn_weights)
attn_weights, attn_weights.shape

(tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
           [0.0000, 1.1297, 0.0000, 0.0000, 0.0000, 0.0000],
           [0.0000, 0.7242, 0.0000, 0.0000, 0.0000, 0.0000],
           [0.4602, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
           [0.3910, 0.0000, 0.4204, 0.0000, 0.3728, 0.0000],
           [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
 
          [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
           [0.5273, 0.7395, 0.0000, 0.0000, 0.0000, 0.0000],
           [0.0000, 0.5540, 0.5513, 0.0000, 0.0000, 0.0000],
           [0.3770, 0.4549, 0.0000, 0.3578, 0.0000, 0.0000],
           [0.3124, 0.3932, 0.3909, 0.0000, 0.0000, 0.0000]]],
 
 
         [[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
           [0.8703, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
           [0.4602, 0.0000, 0.5343, 0.4668, 0.0000, 0.0000],
           [0.3

In [54]:
context_vec = (attn_weights @ values).transpose(1, 2)
context_vec.shape

torch.Size([2, 6, 2, 1])

In [60]:
context_vec = context_vec.contiguous().view(b, num_tokens, d_out)
context_vec = out_proj(context_vec)
context_vec.shape

torch.Size([2, 6, 2])